In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>")) #makes the display bars longer and spread out 100% across the screen width

In [2]:
from vivarium import Artifact
import gbd_mapping as gbd
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from db_queries import get_ids, get_outputs
import scipy.stats

# These are final runs for FALL model 5a and 5b


In [3]:
%cd /share/costeffectiveness/results/vivarium_gates_bep/fall_2020/5_a/india/2020_10_16_08_59_11/count_data/
%ls

!whoami
!date

/ihme/costeffectiveness/results/vivarium_gates_bep/fall_2020/5_a/india/2020_10_16_08_59_11/count_data
birth_weight.csv    deaths.csv           population.csv         ylds.csv
birth_weight.hdf    deaths.hdf           population.hdf         ylds.hdf
cgf_categories.csv  gestational_age.csv  state_person_time.csv  ylls.csv
cgf_categories.hdf  gestational_age.hdf  state_person_time.hdf  ylls.hdf
cgf_z_scores.csv    person_time.csv      transition_count.csv
cgf_z_scores.hdf    person_time.hdf      transition_count.hdf
nicoly
Mon Oct 19 22:00:01 PDT 2020


In [4]:
#scenarios in each of the runs

#run1 (baseline, mmn, bep_hd_universal_i & targeted_i)
#run2 (bep_ce_universal & targeted)
#run3 (bep_ce_universal & targeted)
#run4 (baseline, mmn, bep_hd_universal_iii & targeted_iii)

output_dirs5a = ['/share/costeffectiveness/results/vivarium_gates_bep/fall_2020/5_a/india/2020_10_16_08_59_11/count_data/',
                 '/share/costeffectiveness/results/vivarium_gates_bep/fall_2020/5_a/pakistan/2020_10_16_16_09_17/count_data/',
                 '/share/costeffectiveness/results/vivarium_gates_bep/fall_2020/5_a/mali/2020_10_16_16_06_10/count_data/',
                 '/share/costeffectiveness/results/vivarium_gates_bep/fall_2020/5_a/tanzania/2020_10_16_16_12_19/count_data/']

output_dirs5b = ['/share/costeffectiveness/results/vivarium_gates_bep/fall_2020/5_b/india/2020_10_14_14_43_07/count_data/',
                 '/share/costeffectiveness/results/vivarium_gates_bep/fall_2020/5_b/pakistan/2020_10_16_16_19_08/count_data/',
                 '/share/costeffectiveness/results/vivarium_gates_bep/fall_2020/5_b/mali/2020_10_16_16_16_56/count_data/',
                 '/share/costeffectiveness/results/vivarium_gates_bep/fall_2020/5_b/tanzania/2020_10_16_16_22_21/count_data/']

#location labels
locations = ['India','Pakistan','Mali','Tanzania']

#use in sort_values to make custum sort list. I sorted by region. 
sorter = ['India','Pakistan','Mali','Tanzania']
sorterIndex = dict(zip(sorter,range(len(sorter))))

!whoami
!date

nicoly
Mon Oct 19 22:00:02 PDT 2020


# EXPLORING: INDIA by scenario and treatment groups

In [5]:
bw_raw_india = pd.read_hdf('/share/costeffectiveness/results/vivarium_gates_bep/fall_2020/5_a/india/2020_10_16_08_59_11/count_data/birth_weight.hdf')
bw_raw_india.head()

,risk,treatment_group,mother_status,measure,input_draw,scenario,value
0,birth_weight,bep,malnourished,mean,3,baseline,0.000000
1,birth_weight,bep,malnourished,mean,3,bep_ce_scale_up,2901.301431
2,birth_weight,bep,malnourished,mean,3,bep_ce_targeted_scale_up,2901.301431
3,birth_weight,bep,malnourished,mean,3,mmn_scale_up,0.000000
4,birth_weight,bep,malnourished,mean,21,baseline,0.000000


In [6]:
bw_raw_india['risk'].value_counts()

birth_weight    4800
Name: risk, dtype: int64

In [55]:
bw_raw_india['measure'].value_counts()

sd                        1600
proportion_below_2500g    1600
mean                      1600
Name: measure, dtype: int64

In [7]:
pop_raw_india = pd.read_hdf('/share/costeffectiveness/results/vivarium_gates_bep/fall_2020/5_a/india/2020_10_16_08_59_11/count_data/population.hdf')
pop_raw_india.head()

,treatment_group,mother_status,measure,input_draw,scenario,value
0,all,all,total_population,3,baseline,100000.0
1,all,all,total_population,3,bep_ce_scale_up,100000.0
2,all,all,total_population,3,bep_ce_targeted_scale_up,100000.0
3,all,all,total_population,3,mmn_scale_up,100000.0
4,all,all,total_population,21,baseline,100000.0


In [8]:
pop_raw_india['treatment_group'].value_counts()

all     1000
bep      400
none     400
ifa      400
mmn      400
Name: treatment_group, dtype: int64

In [9]:
pop_raw_india['measure'].value_counts()

total_population              1800
total_population_dead          200
total_population_living        200
total_population_untracked     200
total_population_tracked       200
Name: measure, dtype: int64

In [10]:
pop_raw_india['mother_status'].value_counts()

all             1000
malnourished     800
normal           800
Name: mother_status, dtype: int64

In [11]:
pop_raw_india['scenario'].value_counts()

baseline                    650
mmn_scale_up                650
bep_ce_scale_up             650
bep_ce_targeted_scale_up    650
Name: scenario, dtype: int64

# ================================================================

# GETTING mean BW with 95% CIs for BASELINE

### OVERALL 

In [48]:
#RUNs appended 
# left out run 3 because ii correlation coefficient very similar to iii

overall_bw = pd.DataFrame()

for i in list(range(len(output_dirs5a))):
    
   #birthweight
    bw_001 = pd.read_hdf(output_dirs5a[i] + 'birth_weight.hdf').reset_index().sort_values(by=['risk','measure','scenario','treatment_group','mother_status']).rename(columns={'value':'birthweight'})
    bw_002 = pd.read_hdf(output_dirs5b[i] + 'birth_weight.hdf').reset_index().sort_values(by=['risk','measure','scenario','treatment_group','mother_status']).rename(columns={'value':'birthweight'})
    bw = bw_001.append([bw_002])
    bw = bw.where(bw['measure'] == 'mean').dropna()

    #population
    pop_001 = pd.read_hdf(output_dirs5a[i] + 'population.hdf').reset_index()
    pop_001 = pop_001.where(pop_001['measure'] == 'total_population').dropna().rename(columns={'value':'total_population'}).drop(columns=['index','measure'])

    pop_002 = pd.read_hdf(output_dirs5b[i] + 'population.hdf').reset_index()
    pop_002 = pop_002.where(pop_002['measure'] == 'total_population').dropna().rename(columns={'value':'total_population'}).drop(columns=['index','measure'])

    pop = pop_001.append([pop_002])   

    #merging population and birthweigth 
    data = bw.merge(pop, right_on=['scenario','treatment_group','mother_status','input_draw'], left_on=['scenario','treatment_group','mother_status','input_draw'])
    data['pop_x_bw'] = data['birthweight'] * data['total_population']  

    total_bw = data.groupby(['input_draw', 'scenario']).pop_x_bw.sum().reset_index()
    total_bw['total_pop'] = 100_000
    total_bw['overall_mean_bw'] = total_bw['pop_x_bw']/total_bw['total_pop']
    
    total_bw = total_bw.groupby(['scenario']).overall_mean_bw.describe(percentiles=[.025, 0.975]).filter(['mean', '2.5%', '97.5%']).reset_index()     
    total_bw['location'] = locations[i]
    
    overall_bw = overall_bw.append(total_bw)   
    
overall_bw[overall_bw['scenario'].isin(['baseline'])]


,scenario,mean,2.5%,97.5%,location
0,baseline,2886.520761,2877.787260,2896.927079,india
0,baseline,2839.551113,2814.771295,2868.197220,pakistan
0,baseline,2930.028115,2895.916765,3006.195159,mali
0,baseline,3274.755314,3189.985105,3321.339572,tanzania


# MALNOURISHED 

In [50]:
overall_mean_bw_mal  = pd.DataFrame()

for i in list(range(len(output_dirs5a))):

#birthweight for MALNOURISED

    bw_001 = pd.read_hdf(output_dirs5a[i] + 'birth_weight.hdf').reset_index().sort_values(by=['risk','measure','scenario','treatment_group','mother_status']).rename(columns={'value':'birthweight'})
    bw_002 = pd.read_hdf(output_dirs5b[i] + 'birth_weight.hdf').reset_index().sort_values(by=['risk','measure','scenario','treatment_group','mother_status']).rename(columns={'value':'birthweight'})
    bw = bw_001.append([bw_002])
    bw = bw.where((bw['measure'] == 'mean') & (bw['mother_status'] == 'malnourished')).dropna()

    #population
    pop_001 = pd.read_hdf(output_dirs5a[i] + 'population.hdf').reset_index()
    pop_001 = pop_001.where(pop_001['measure'] == 'total_population').dropna().rename(columns={'value':'total_population'}).drop(columns=['index','measure'])

    pop_002 = pd.read_hdf(output_dirs5b[i] + 'population.hdf').reset_index()
    pop_002 = pop_002.where(pop_002['measure'] == 'total_population').dropna().rename(columns={'value':'total_population'}).drop(columns=['index','measure'])

    pop = pop_001.append([pop_002])   
    pop = pop.where(pop['mother_status'] == 'malnourished').dropna()

    #merging population and birthweigth 
    data = bw.merge(pop, right_on=['scenario','treatment_group','mother_status','input_draw'], left_on=['scenario','treatment_group','mother_status','input_draw'])
    data['pop_x_bw'] = data['birthweight'] * data['total_population']  

    total_bw = data.groupby(['input_draw', 'scenario']).pop_x_bw.sum().reset_index()
    total_malpop = data.groupby(['input_draw', 'scenario']).total_population.sum().reset_index()

    mean_bw_mal = pd.merge(total_bw, total_malpop, left_on = ['input_draw', 'scenario'], right_on =['input_draw', 'scenario'])

    mean_bw_mal['overall_mean_bw'] = mean_bw_mal['pop_x_bw']/mean_bw_mal['total_population']
    #get 95%CI
    mean_bw_mal = mean_bw_mal.groupby(['scenario']).overall_mean_bw.describe(percentiles=[.025, 0.975]).filter(['mean', '2.5%', '97.5%']).reset_index()     

    mean_bw_mal['location'] = locations[i]
    
    overall_mean_bw_mal = overall_mean_bw_mal.append(mean_bw_mal)   
    
overall_mean_bw_mal[overall_mean_bw_mal['scenario'].isin(['baseline'])]
 


,scenario,mean,2.5%,97.5%,location
0,baseline,2759.100348,2717.176456,2811.614610,india
0,baseline,2701.817861,2626.179992,2754.877675,pakistan
0,baseline,2795.151256,2739.304021,2865.298192,mali
0,baseline,3141.589553,3052.252229,3247.349144,tanzania


# NORMAL 

In [54]:
overall_mean_bw_norm  = pd.DataFrame()

for i in list(range(len(output_dirs5a))):

#birthweight for normNOURISED

    bw_001 = pd.read_hdf(output_dirs5a[i] + 'birth_weight.hdf').reset_index().sort_values(by=['risk','measure','scenario','treatment_group','mother_status']).rename(columns={'value':'birthweight'})
    bw_002 = pd.read_hdf(output_dirs5b[i] + 'birth_weight.hdf').reset_index().sort_values(by=['risk','measure','scenario','treatment_group','mother_status']).rename(columns={'value':'birthweight'})
    bw = bw_001.append([bw_002])
    bw = bw.where((bw['measure'] == 'mean') & (bw['mother_status'] == 'normal')).dropna()

    #population
    pop_001 = pd.read_hdf(output_dirs5a[i] + 'population.hdf').reset_index()
    pop_001 = pop_001.where(pop_001['measure'] == 'total_population').dropna().rename(columns={'value':'total_population'}).drop(columns=['index','measure'])

    pop_002 = pd.read_hdf(output_dirs5b[i] + 'population.hdf').reset_index()
    pop_002 = pop_002.where(pop_002['measure'] == 'total_population').dropna().rename(columns={'value':'total_population'}).drop(columns=['index','measure'])

    pop = pop_001.append([pop_002])   
    pop = pop.where(pop['mother_status'] == 'normal').dropna()

    #merging population and birthweigth 
    data = bw.merge(pop, right_on=['scenario','treatment_group','mother_status','input_draw'], left_on=['scenario','treatment_group','mother_status','input_draw'])
    data['pop_x_bw'] = data['birthweight'] * data['total_population']  

    total_bw = data.groupby(['input_draw', 'scenario']).pop_x_bw.sum().reset_index()
    total_normpop = data.groupby(['input_draw', 'scenario']).total_population.sum().reset_index()

    mean_bw_norm = pd.merge(total_bw, total_normpop, left_on = ['input_draw', 'scenario'], right_on =['input_draw', 'scenario'])

    mean_bw_norm['overall_mean_bw'] = mean_bw_norm['pop_x_bw']/mean_bw_norm['total_population']
    #get 95%CI
    mean_bw_norm = mean_bw_norm.groupby(['scenario']).overall_mean_bw.describe(percentiles=[.025, 0.975]).filter(['mean', '2.5%', '97.5%']).reset_index()     

    mean_bw_norm['location'] = locations[i]
    
    overall_mean_bw_norm = overall_mean_bw_norm.append(mean_bw_norm)   
    
overall_mean_bw_norm[overall_mean_bw_norm['scenario'].isin(['baseline'])]
 


,scenario,mean,2.5%,97.5%,location
0,baseline,2912.483048,2896.833617,2933.105568,india
0,baseline,2856.186172,2834.917880,2889.157566,pakistan
0,baseline,2945.625323,2908.736835,3025.210393,mali
0,baseline,3288.890966,3202.380498,3335.672819,tanzania


# ------------------------------------------------------------------

# GETTING LBW categories <2500 with 95% CIs for BASELINE

### OVERALL 

In [58]:
#RUNs appended 
# left out run 3 because ii correlation coefficient very similar to iii

overall_cat_bw = pd.DataFrame()

for i in list(range(len(output_dirs5a))):
    
   #birthweight
    bw_001 = pd.read_hdf(output_dirs5a[i] + 'birth_weight.hdf').reset_index().sort_values(by=['risk','measure','scenario','treatment_group','mother_status']).rename(columns={'value':'proportion_below_2500g'})
    bw_002 = pd.read_hdf(output_dirs5b[i] + 'birth_weight.hdf').reset_index().sort_values(by=['risk','measure','scenario','treatment_group','mother_status']).rename(columns={'value':'proportion_below_2500g'})
    bw = bw_001.append([bw_002])
    bw = bw.where(bw['measure'] == 'proportion_below_2500g').dropna()

    #population
    pop_001 = pd.read_hdf(output_dirs5a[i] + 'population.hdf').reset_index()
    pop_001 = pop_001.where(pop_001['measure'] == 'total_population').dropna().rename(columns={'value':'total_population'}).drop(columns=['index','measure'])

    pop_002 = pd.read_hdf(output_dirs5b[i] + 'population.hdf').reset_index()
    pop_002 = pop_002.where(pop_002['measure'] == 'total_population').dropna().rename(columns={'value':'total_population'}).drop(columns=['index','measure'])

    pop = pop_001.append([pop_002])   

    #merging population and birthweigth 
    data = bw.merge(pop, right_on=['scenario','treatment_group','mother_status','input_draw'], left_on=['scenario','treatment_group','mother_status','input_draw'])
    data['pop_x_bw'] = data['proportion_below_2500g'] * data['total_population']  

    total_bw = data.groupby(['input_draw', 'scenario']).pop_x_bw.sum().reset_index()
    total_bw['total_pop'] = 100_000
    total_bw['overall_cat_bw'] = total_bw['pop_x_bw']/total_bw['total_pop']
    
    total_bw = total_bw.groupby(['scenario']).overall_cat_bw.describe(percentiles=[.025, 0.975]).filter(['mean', '2.5%', '97.5%']).reset_index()     
    total_bw['location'] = locations[i]
    
    overall_cat_bw = overall_cat_bw.append(total_bw)   
    
overall_cat_bw[overall_cat_bw['scenario'].isin(['baseline'])]


,scenario,mean,2.5%,97.5%,location
0,baseline,0.216402,0.208702,0.224695,india
0,baseline,0.237566,0.218212,0.258459,pakistan
0,baseline,0.179298,0.152976,0.197422,mali
0,baseline,0.075179,0.062037,0.098507,tanzania


# ================================================================

# ****** -- CHECK birthweight shifts by scenario -- ******


***************************************************************************************************


In [15]:
#RUNs appended 
# left out run 3 because ii correlation coefficient very similar to iii

master_bw= pd.DataFrame()

for i in list(range(len(output_dirs5a))):
    
    #birthweight
    bw_001 = pd.read_hdf(output_dirs5a[i] + 'birth_weight.hdf').reset_index().sort_values(by=['risk','measure','scenario','treatment_group','mother_status']).rename(columns={'value':'birthweight'})
    bw_002 = pd.read_hdf(output_dirs5b[i] + 'birth_weight.hdf').reset_index().sort_values(by=['risk','measure','scenario','treatment_group','mother_status']).rename(columns={'value':'birthweight'})

    bw = bw_001.append([bw_002])
  
    
    #population
    pop_001 = pd.read_hdf(output_dirs5a[i] + 'population.hdf').reset_index()
    pop_001 = pop_001.where(pop_001['measure'] == 'total_population').dropna().rename(columns={'value':'total_population'}).drop(columns=['index','measure'])
    
    pop_002 = pd.read_hdf(output_dirs5b[i] + 'population.hdf').reset_index()
    pop_002 = pop_002.where(pop_002['measure'] == 'total_population').dropna().rename(columns={'value':'total_population'}).drop(columns=['index','measure'])
    
    pop = pop_001.append([pop_002])   
    
    data = bw.merge(pop, right_on=['scenario','treatment_group','mother_status','input_draw'], left_on=['scenario','treatment_group','mother_status','input_draw'])
    data = data.groupby(['risk','measure','scenario','treatment_group','mother_status']).mean().drop(columns=['input_draw','index']).reset_index() #average over input draws
    data['pop_x_bw'] = data['birthweight'] * data['total_population']
    data['location'] = locations[i]
    master_bw = master_bw.append(data)
    
master_bw.head()


,risk,measure,scenario,treatment_group,mother_status,birthweight,total_population,pop_x_bw,location
0,birth_weight,mean,baseline,bep,malnourished,0.000000,0.00,0.000000e+00,India
1,birth_weight,mean,baseline,bep,normal,0.000000,0.00,0.000000e+00,India
2,birth_weight,mean,baseline,ifa,malnourished,2799.256953,5770.46,1.615300e+07,India
3,birth_weight,mean,baseline,ifa,normal,2951.942669,28262.46,8.342916e+07,India
4,birth_weight,mean,baseline,mmn,malnourished,0.000000,0.00,0.000000e+00,India


## *BASELINE

In [16]:
#to get the dataframe we want from run1 (BASELINE):
#INDIA

india_bw_baseline = master_bw[master_bw['location'].isin(['India']) & master_bw['measure'].isin(['mean']) & master_bw['scenario'].isin(['baseline']) & master_bw['treatment_group'].isin(['ifa','none'])]
india_bwsd_baseline = master_bw[master_bw['location'].isin(['India']) & master_bw['measure'].isin(['sd']) & master_bw['scenario'].isin(['baseline']) & master_bw['treatment_group'].isin(['ifa','none'])]

print(india_bw_baseline['pop_x_bw'].sum()/100_000)
print(india_bwsd_baseline['pop_x_bw'].sum()/100_000)
india_bw_baseline

2886.4548416987677
691.6361189407631


,risk,measure,scenario,treatment_group,mother_status,birthweight,total_population,pop_x_bw,location
2,birth_weight,mean,baseline,ifa,malnourished,2799.256953,5770.46,1.615300e+07,India
3,birth_weight,mean,baseline,ifa,normal,2951.942669,28262.46,8.342916e+07,India
6,birth_weight,mean,baseline,none,malnourished,2738.229854,11166.04,3.057518e+07,India
7,birth_weight,mean,baseline,none,normal,2892.064424,54801.04,1.584881e+08,India


In [17]:
#pakistan

pakistan_bw_baseline = master_bw[master_bw['location'].isin(['Pakistan']) & master_bw['measure'].isin(['mean']) & master_bw['scenario'].isin(['baseline']) & master_bw['treatment_group'].isin(['ifa','none'])]
pakistan_bwsd_baseline = master_bw[master_bw['location'].isin(['Pakistan']) & master_bw['measure'].isin(['sd']) & master_bw['scenario'].isin(['baseline']) & master_bw['treatment_group'].isin(['ifa','none'])]

print(pakistan_bw_baseline['pop_x_bw'].sum()/100_000)
print(pakistan_bwsd_baseline['pop_x_bw'].sum()/100_000)
pakistan_bw_baseline 

2839.554174321471
698.0559711192248


,risk,measure,scenario,treatment_group,mother_status,birthweight,total_population,pop_x_bw,location
2,birth_weight,mean,baseline,ifa,malnourished,2747.219664,2668.90,7.332055e+06,Pakistan
3,birth_weight,mean,baseline,ifa,normal,2900.512271,22041.16,6.393066e+07,Pakistan
6,birth_weight,mean,baseline,none,malnourished,2686.869627,8109.32,2.178869e+07,Pakistan
7,birth_weight,mean,baseline,none,normal,2841.653176,67180.62,1.909040e+08,Pakistan


In [18]:
#MALI

mali_bw_baseline = master_bw[master_bw['location'].isin(['Mali']) & master_bw['measure'].isin(['mean']) & master_bw['scenario'].isin(['baseline']) & master_bw['treatment_group'].isin(['ifa','none'])]
mali_bwsd_baseline = master_bw[master_bw['location'].isin(['Mali']) & master_bw['measure'].isin(['sd']) & master_bw['scenario'].isin(['baseline']) & master_bw['treatment_group'].isin(['ifa','none'])]
mali_bw_baseline 

print(mali_bw_baseline['pop_x_bw'].sum()/100_000)
print(mali_bwsd_baseline['pop_x_bw'].sum()/100_000)
mali_bw_baseline 

2930.0312188464914
660.1045452013743


,risk,measure,scenario,treatment_group,mother_status,birthweight,total_population,pop_x_bw,location
2,birth_weight,mean,baseline,ifa,malnourished,2840.521393,2421.32,6.877811e+06,Mali
3,birth_weight,mean,baseline,ifa,normal,2991.942211,20865.44,6.242819e+07,Mali
6,birth_weight,mean,baseline,none,malnourished,2781.355908,7952.18,2.211784e+07,Mali
7,birth_weight,mean,baseline,none,normal,2931.590600,68761.06,2.015793e+08,Mali


In [19]:
#Tanzania

tanzania_bw_baseline = master_bw[master_bw['location'].isin(['Tanzania']) & master_bw['measure'].isin(['mean']) & master_bw['scenario'].isin(['baseline']) & master_bw['treatment_group'].isin(['ifa','none'])]
tanzania_bwsd_baseline = master_bw[master_bw['location'].isin(['Tanzania']) & master_bw['measure'].isin(['sd']) & master_bw['scenario'].isin(['baseline']) & master_bw['treatment_group'].isin(['ifa','none'])]
tanzania_bw_baseline 

print(tanzania_bw_baseline['pop_x_bw'].sum()/100_000)
print(tanzania_bwsd_baseline['pop_x_bw'].sum()/100_000)
tanzania_bw_baseline 

3274.7615355859452
636.1296742217747


,risk,measure,scenario,treatment_group,mother_status,birthweight,total_population,pop_x_bw,location
2,birth_weight,mean,baseline,ifa,malnourished,3188.098333,2014.68,6.422998e+06,Tanzania
3,birth_weight,mean,baseline,ifa,normal,3335.282096,18957.34,6.322808e+07,Tanzania
6,birth_weight,mean,baseline,none,malnourished,3129.134269,7553.38,2.363554e+07,Tanzania
7,birth_weight,mean,baseline,none,normal,3276.542138,71474.60,2.341895e+08,Tanzania


# SCENARIOS 

## MMN SCALE-UP

In [20]:
#to get the dataframe we want from run1-TEST(MMN_SCALE_UP):

bw_mmn = master_bw[master_bw['location'].isin(['India']) & master_bw['measure'].isin(['mean']) & master_bw['scenario'].isin(['mmn_scale_up']) & master_bw['treatment_group'].isin(['mmn','none'])]
bw_mmn


,risk,measure,scenario,treatment_group,mother_status,birthweight,total_population,pop_x_bw,location
44,birth_weight,mean,mmn_scale_up,mmn,malnourished,2842.242816,13444.32,3.821202e+07,India
45,birth_weight,mean,mmn_scale_up,mmn,normal,2996.434218,65971.26,1.976785e+08,India
46,birth_weight,mean,mmn_scale_up,none,malnourished,2741.605354,3492.18,9.574179e+06,India
47,birth_weight,mean,mmn_scale_up,none,normal,2891.957944,17092.24,4.943004e+07,India


## BEP HD SCALE-UP UNIVERSAL

In [21]:
#to get the dataframe we want from run1-TEST (BEP_HD_SCALE_UP iii):

bw_bep_hd_universal = master_bw[master_bw['location'].isin(['India']) & master_bw['measure'].isin(['mean']) & master_bw['scenario'].isin(['bep_hd_scale_up']) & master_bw['treatment_group'].isin(['bep','none'])]
bw_bep_hd_universal

,risk,measure,scenario,treatment_group,mother_status,birthweight,total_population,pop_x_bw,location
24,birth_weight,mean,bep_hd_scale_up,bep,malnourished,2942.209307,13444.32,3.955600e+07,India
25,birth_weight,mean,bep_hd_scale_up,bep,normal,3071.400709,65971.26,2.026242e+08,India
30,birth_weight,mean,bep_hd_scale_up,none,malnourished,2741.605354,3492.18,9.574179e+06,India
31,birth_weight,mean,bep_hd_scale_up,none,normal,2891.957944,17092.24,4.943004e+07,India


## BEP HD TARGETED SCALE UP

In [22]:
#to get the dataframe we want from  (BEP_HD_TARGETED_SCALE_UP iii):

bw_bep_hd_targeted= master_bw[master_bw['location'].isin(['India']) & master_bw['measure'].isin(['mean']) & master_bw['scenario'].isin(['bep_hd_targeted_scale_up']) & master_bw['treatment_group'].isin(['bep','mmn','none'])]
bw_bep_hd_targeted

,risk,measure,scenario,treatment_group,mother_status,birthweight,total_population,pop_x_bw,location
32,birth_weight,mean,bep_hd_targeted_scale_up,bep,malnourished,2942.209307,13444.32,3.955600e+07,India
33,birth_weight,mean,bep_hd_targeted_scale_up,bep,normal,0.000000,0.00,0.000000e+00,India
36,birth_weight,mean,bep_hd_targeted_scale_up,mmn,malnourished,0.000000,0.00,0.000000e+00,India
37,birth_weight,mean,bep_hd_targeted_scale_up,mmn,normal,2996.434218,65971.26,1.976785e+08,India
38,birth_weight,mean,bep_hd_targeted_scale_up,none,malnourished,2741.605354,3492.18,9.574179e+06,India
39,birth_weight,mean,bep_hd_targeted_scale_up,none,normal,2891.957944,17092.24,4.943004e+07,India


## BEP CE UNIVERSAL SCALE UP

In [23]:
!whoami
!date

nicoly
Mon Oct 19 22:00:04 PDT 2020


In [24]:
#to get the dataframe we want from run1-TEST (BEP_HD_SCALE_UP iii):

bw_bep_ce_universal = master_bw[master_bw['location'].isin(['India']) & master_bw['measure'].isin(['mean']) & master_bw['scenario'].isin(['bep_ce_scale_up']) & master_bw['treatment_group'].isin(['bep','none'])]
bw_bep_ce_universal

,risk,measure,scenario,treatment_group,mother_status,birthweight,total_population,pop_x_bw,location
8,birth_weight,mean,bep_ce_scale_up,bep,malnourished,2909.094405,13444.32,3.911080e+07,India
9,birth_weight,mean,bep_ce_scale_up,bep,normal,3012.887889,65971.26,1.987640e+08,India
14,birth_weight,mean,bep_ce_scale_up,none,malnourished,2741.605354,3492.18,9.574179e+06,India
15,birth_weight,mean,bep_ce_scale_up,none,normal,2891.957944,17092.24,4.943004e+07,India


## BEP CE TARGETED SCALE UP

In [25]:
!whoami
!date

nicoly
Mon Oct 19 22:00:04 PDT 2020


In [26]:
#to get the dataframe we want from  (BEP_HD_TARGETED_SCALE_UP iii):

bw_bep_ce_targeted= master_bw[master_bw['location'].isin(['India']) & master_bw['measure'].isin(['mean']) & master_bw['scenario'].isin(['bep_ce_targeted_scale_up']) & master_bw['treatment_group'].isin(['bep','mmn','none'])]
bw_bep_ce_targeted

,risk,measure,scenario,treatment_group,mother_status,birthweight,total_population,pop_x_bw,location
16,birth_weight,mean,bep_ce_targeted_scale_up,bep,malnourished,2909.094405,13444.32,3.911080e+07,India
17,birth_weight,mean,bep_ce_targeted_scale_up,bep,normal,0.000000,0.00,0.000000e+00,India
20,birth_weight,mean,bep_ce_targeted_scale_up,mmn,malnourished,0.000000,0.00,0.000000e+00,India
21,birth_weight,mean,bep_ce_targeted_scale_up,mmn,normal,2996.434218,65971.26,1.976785e+08,India
22,birth_weight,mean,bep_ce_targeted_scale_up,none,malnourished,2741.605354,3492.18,9.574179e+06,India
23,birth_weight,mean,bep_ce_targeted_scale_up,none,normal,2891.957944,17092.24,4.943004e+07,India


# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~




# ======== VALIDATION OF BASELINE WITH GBD ARTIFACT FOR BIRTHWEIGHT ===============

# 1. get population sex distribution from artifact

we collapsed over sex in out outputs.hdf (we didn't ask the engineers to give us sex stratifications
however, the 

In [27]:
# get population birth sex ratio from artifact data

locations = ['india', 'pakistan', 'mali', 'tanzania']

def load_artifact_sr():
    tab = pd.DataFrame()
    for i in locations:
        art = Artifact('/share/costeffectiveness/artifacts/vivarium_gates_bep/' + i + '.hdf', filter_terms=['year_start == 2017', 'age_start == 0'])
        print(i)
        sr = art.load('population.structure').reset_index()
        sr = sr.rename(columns={'value':'pop'})
        sr['total_pop'] = sr['pop'][0] + sr['pop'][1]
        sr['sex_ratio'] = sr['pop'] / sr['total_pop']
        tab=tab.append(sr)
    return tab


In [28]:
sex_ratio = load_artifact_sr()
sex_ratio

india
pakistan
mali
tanzania


,location,sex,age_start,age_end,year_start,year_end,pop,total_pop,sex_ratio
0,India,Female,0.0,0.019178,2017,2018,225403.558880,473130.847643,0.476409
1,India,Male,0.0,0.019178,2017,2018,247727.288763,473130.847643,0.523591
0,Pakistan,Female,0.0,0.019178,2017,2018,53994.159557,112567.291362,0.479661
1,Pakistan,Male,0.0,0.019178,2017,2018,58573.131805,112567.291362,0.520339
0,Mali,Female,0.0,0.019178,2017,2018,8184.729977,16778.605515,0.487808
1,Mali,Male,0.0,0.019178,2017,2018,8593.875539,16778.605515,0.512192
0,Tanzania,Female,0.0,0.019178,2017,2018,18631.222060,37630.501139,0.495110
1,Tanzania,Male,0.0,0.019178,2017,2018,18999.279079,37630.501139,0.504890


# 2. Get LBWSG exposure categories by country FROM GBD (artifact)


from  from Ali's -- LBWSG exposure, rrs, pafs -- noteook

In [29]:
def get_lbwsg_exposure_and_rr_data(location):
    art = pd.HDFStore('/share/costeffectiveness/artifacts/vivarium_gates_bep/' + location + '.hdf')
    keys = art.keys()
    exposure_index = art.get('/risk_factor/low_birth_weight_and_short_gestation/exposure/index')
    exposure_data = art.get('/risk_factor/low_birth_weight_and_short_gestation/exposure/draw_0')
    exposure = exposure_index.merge(exposure_data, right_index=True, left_index=True)
    exposure = (exposure.rename(columns={'draw_0':'exposure'})
                .where(exposure['year_start'] == 2017)
                .where(exposure['age_start'] == 0).dropna()) #these are the 0-7 day years olds
    
    relative_risk_index = art.get('/risk_factor/low_birth_weight_and_short_gestation/relative_risk/index')
    relative_risk_data = art.get('/risk_factor/low_birth_weight_and_short_gestation/relative_risk/draw_0')
    relative_risk = relative_risk_index.merge(relative_risk_data, right_index=True, left_index=True)
    relative_risk = (relative_risk.rename(columns={'draw_0':'relative_risk'})
                .where(relative_risk['year_start'] == 2017)
                .where(relative_risk['age_start'] < 1.92e-02).dropna())
    
    data = exposure.merge(relative_risk, right_on=['location','sex','age_start','age_end',
                                                   'year_start','year_end','parameter'],
                         left_on=['location','sex','age_start','age_end','year_start','year_end','parameter'])
    return data


In [30]:
pwd

'/ihme/costeffectiveness/results/vivarium_gates_bep/fall_2020/5_a/india/2020_10_16_08_59_11/count_data'

Defining the lbw tables by country

In [31]:
#total of 228 rows

lbwart_india= get_lbwsg_exposure_and_rr_data('india')
lbwart_pakistan = get_lbwsg_exposure_and_rr_data('pakistan')
lbwart_mali = get_lbwsg_exposure_and_rr_data('mali')
lbwart_tanzania = get_lbwsg_exposure_and_rr_data('tanzania')
#lbw_mali.to_csv('/ihme/homes/nicoly/notebooks/Project BEP/Validation/lbw_mali')
lbwart_mali.head()

,location,sex,age_start,age_end,year_start,year_end,parameter,exposure,affected_entity,affected_measure,relative_risk
0,Mali,Female,0.0,0.019178,2017.0,2018.0,cat10,0.002128,all,excess_mortality_rate,968.702386
1,Mali,Female,0.0,0.019178,2017.0,2018.0,cat106,0.001562,all,excess_mortality_rate,171.751901
2,Mali,Female,0.0,0.019178,2017.0,2018.0,cat11,0.001454,all,excess_mortality_rate,513.870580
3,Mali,Female,0.0,0.019178,2017.0,2018.0,cat116,0.003774,all,excess_mortality_rate,182.396469
4,Mali,Female,0.0,0.019178,2017.0,2018.0,cat117,0.014307,all,excess_mortality_rate,61.431240


In [32]:
lbwart_mali.relative_risk.nunique()
#print(len(relativerisk))

111

In [33]:
lbwart_mali.parameter.nunique()
#print(len(cat))

57

In [34]:
sex = lbwart_mali.sex.unique()
#print(len(cat))
sex

array(['Female', 'Male'], dtype=object)

In [35]:
sex = lbwart_mali.age_start.unique()
#print(len(cat))
sex

array([0.])

## Get continuous birthweight by category using function from Abie's notebook

In [36]:
categories = gbd.risk_factors.low_birth_weight_and_short_gestation.categories

In [37]:
# make dataframe that maps from each category to gestatation_age_start,
# gestatation_age_end, birth_weight_start, and birth weight end

s = pd.Series(categories.to_dict())


In [38]:
# map mean birthweight to cat

def ga_bw_list(cat):
    numbers_only = cat.replace('Birth prevalence - [', '') \
                    .replace(',', '')\
                    .replace(') wks [', ' ')\
                    .replace(') g', '')
    numbers_only = numbers_only.split()
    numbers_only = [int(numbers_only[i]) for i in range(4)]
    
    return numbers_only

i = np.random.randint(len(s))
s.iloc[i], ga_bw_list(s.iloc[i])

('Birth prevalence - [32, 34) wks, [2000, 2500) g', [32, 34, 2000, 2500])

In [39]:
#test equation

l = ga_bw_list(s['cat10'])
(l[2] + l[3]) / 2

750.0

In [40]:
lbwart_mali.head()

,location,sex,age_start,age_end,year_start,year_end,parameter,exposure,affected_entity,affected_measure,relative_risk
0,Mali,Female,0.0,0.019178,2017.0,2018.0,cat10,0.002128,all,excess_mortality_rate,968.702386
1,Mali,Female,0.0,0.019178,2017.0,2018.0,cat106,0.001562,all,excess_mortality_rate,171.751901
2,Mali,Female,0.0,0.019178,2017.0,2018.0,cat11,0.001454,all,excess_mortality_rate,513.870580
3,Mali,Female,0.0,0.019178,2017.0,2018.0,cat116,0.003774,all,excess_mortality_rate,182.396469
4,Mali,Female,0.0,0.019178,2017.0,2018.0,cat117,0.014307,all,excess_mortality_rate,61.431240


# ---------------------------------------------------------------------------------------------------------------------

# OVERALL BIRTHWEIGHT COMPARISONS FOR V & V (artifact and baseline model)

In [41]:
#sex ratio from artficat data

sex_ratio

,location,sex,age_start,age_end,year_start,year_end,pop,total_pop,sex_ratio
0,India,Female,0.0,0.019178,2017,2018,225403.558880,473130.847643,0.476409
1,India,Male,0.0,0.019178,2017,2018,247727.288763,473130.847643,0.523591
0,Pakistan,Female,0.0,0.019178,2017,2018,53994.159557,112567.291362,0.479661
1,Pakistan,Male,0.0,0.019178,2017,2018,58573.131805,112567.291362,0.520339
0,Mali,Female,0.0,0.019178,2017,2018,8184.729977,16778.605515,0.487808
1,Mali,Male,0.0,0.019178,2017,2018,8593.875539,16778.605515,0.512192
0,Tanzania,Female,0.0,0.019178,2017,2018,18631.222060,37630.501139,0.495110
1,Tanzania,Male,0.0,0.019178,2017,2018,18999.279079,37630.501139,0.504890


## INDIA

note: sd from output.hdf are the individual sd , while the measure of spread in the artficat are the standard errors...hence, dont think there is a easy way to compare sds. 

In [42]:
# add mean birthweight to india 

# lbw_india.mean_bw = ga_bw_list(s[lbw_india.parameter])
lbwart_india['mean_bw'] = lbwart_india['parameter'].apply(lambda x: (ga_bw_list(s[x])[2]+ga_bw_list(s[x])[3])/2)
lbwart_india['sex_ratio'] = np.where(lbwart_india['sex']== 'Female', 0.476409, 1-0.476409)
lbwart_india['weighted_bw'] = lbwart_india['exposure'] * lbwart_india['mean_bw'] * lbwart_india['sex_ratio']

print('overall mean birthweight from artifact baseline (india) = ', + lbwart_india['weighted_bw'].sum())
print('overall mean birthweight from model baseline (india) = ', + india_bw_baseline['pop_x_bw'].sum()/100_000, )
print('overall mean sd of birthweight from model baseline (india) = ', + india_bwsd_baseline['pop_x_bw'].sum()/100_000)

lbwart_india.head()

overall mean birthweight from artifact baseline (india) =  2882.5449288053464
overall mean birthweight from model baseline (india) =  2886.4548416987677
overall mean sd of birthweight from model baseline (india) =  691.6361189407631


,location,sex,age_start,age_end,year_start,year_end,parameter,exposure,affected_entity,affected_measure,relative_risk,mean_bw,sex_ratio,weighted_bw
0,India,Female,0.0,0.019178,2017.0,2018.0,cat10,0.000590,all,excess_mortality_rate,968.702386,750.0,0.476409,0.210744
1,India,Female,0.0,0.019178,2017.0,2018.0,cat106,0.003773,all,excess_mortality_rate,171.751901,1250.0,0.476409,2.246593
2,India,Female,0.0,0.019178,2017.0,2018.0,cat11,0.000582,all,excess_mortality_rate,513.870580,750.0,0.476409,0.208009
3,India,Female,0.0,0.019178,2017.0,2018.0,cat116,0.005572,all,excess_mortality_rate,182.396469,1250.0,0.476409,3.318133
4,India,Female,0.0,0.019178,2017.0,2018.0,cat117,0.017522,all,excess_mortality_rate,61.431240,1750.0,0.476409,14.608434


## PAKISTAN

In [43]:
# add mean birthweight to pakistan 

# lbw_pakistan.mean_bw = ga_bw_list(s[lbw_pakistan.parameter])
lbwart_pakistan['mean_bw'] = lbwart_pakistan['parameter'].apply(lambda x: (ga_bw_list(s[x])[2]+ga_bw_list(s[x])[3])/2)
lbwart_pakistan['sex_ratio'] = np.where(lbwart_pakistan['sex']== 'Female', 0.479661, 1-0.479661)
lbwart_pakistan['weighted_bw'] = lbwart_pakistan['exposure'] * lbwart_pakistan['mean_bw'] * lbwart_pakistan['sex_ratio']

print('overall mean birthweight from artifact baseline (pakistan) = ', + lbwart_pakistan['weighted_bw'].sum())
print('overall mean birthweight from model baseline (pakistan) = ', + pakistan_bw_baseline['pop_x_bw'].sum()/100_000, )
print('overall mean sd of birthweight from model baseline (pakistan) = ', + pakistan_bwsd_baseline['pop_x_bw'].sum()/100_000)

lbwart_pakistan.head()

overall mean birthweight from artifact baseline (pakistan) =  2840.9029325769707
overall mean birthweight from model baseline (pakistan) =  2839.554174321471
overall mean sd of birthweight from model baseline (pakistan) =  698.0559711192248


,location,sex,age_start,age_end,year_start,year_end,parameter,exposure,affected_entity,affected_measure,relative_risk,mean_bw,sex_ratio,weighted_bw
0,Pakistan,Female,0.0,0.019178,2017.0,2018.0,cat10,0.000000,all,excess_mortality_rate,968.702386,750.0,0.479661,0.000000
1,Pakistan,Female,0.0,0.019178,2017.0,2018.0,cat106,0.004062,all,excess_mortality_rate,171.751901,1250.0,0.479661,2.435586
2,Pakistan,Female,0.0,0.019178,2017.0,2018.0,cat11,0.000542,all,excess_mortality_rate,513.870580,750.0,0.479661,0.195109
3,Pakistan,Female,0.0,0.019178,2017.0,2018.0,cat116,0.005284,all,excess_mortality_rate,182.396469,1250.0,0.479661,3.168183
4,Pakistan,Female,0.0,0.019178,2017.0,2018.0,cat117,0.019681,all,excess_mortality_rate,61.431240,1750.0,0.479661,16.520460


## MALI

In [44]:
# add mean birthweight to mali 

lbwart_mali['mean_bw'] = lbwart_mali['parameter'].apply(lambda x: (ga_bw_list(s[x])[2]+ga_bw_list(s[x])[3])/2)
lbwart_mali['sex_ratio'] = np.where(lbwart_mali['sex']== 'Female', 0.487808, 1-0.487808)
lbwart_mali['weighted_bw'] = lbwart_mali['exposure'] * lbwart_mali['mean_bw'] * lbwart_mali['sex_ratio']

print('overall mean birthweight from artifact baseline (mali) = ', + lbwart_mali['weighted_bw'].sum())
print('overall mean birthweight from model baseline (mali) = ', + mali_bw_baseline['pop_x_bw'].sum()/100_000, )
print('overall mean sd of birthweight from model baseline (mali) = ', + mali_bwsd_baseline['pop_x_bw'].sum()/100_000)

lbwart_mali.head()

overall mean birthweight from artifact baseline (mali) =  2930.2635119015226
overall mean birthweight from model baseline (mali) =  2930.0312188464914
overall mean sd of birthweight from model baseline (mali) =  660.1045452013743


,location,sex,age_start,age_end,year_start,year_end,parameter,exposure,affected_entity,affected_measure,relative_risk,mean_bw,sex_ratio,weighted_bw
0,Mali,Female,0.0,0.019178,2017.0,2018.0,cat10,0.002128,all,excess_mortality_rate,968.702386,750.0,0.487808,0.778673
1,Mali,Female,0.0,0.019178,2017.0,2018.0,cat106,0.001562,all,excess_mortality_rate,171.751901,1250.0,0.487808,0.952270
2,Mali,Female,0.0,0.019178,2017.0,2018.0,cat11,0.001454,all,excess_mortality_rate,513.870580,750.0,0.487808,0.531772
3,Mali,Female,0.0,0.019178,2017.0,2018.0,cat116,0.003774,all,excess_mortality_rate,182.396469,1250.0,0.487808,2.301030
4,Mali,Female,0.0,0.019178,2017.0,2018.0,cat117,0.014307,all,excess_mortality_rate,61.431240,1750.0,0.487808,12.213578


## Tanzania

In [45]:
# add mean birthweight to tanzania 

# lbw_tanzania.mean_bw = ga_bw_list(s[lbw_tanzania.parameter])
lbwart_tanzania['mean_bw'] = lbwart_tanzania['parameter'].apply(lambda x: (ga_bw_list(s[x])[2]+ga_bw_list(s[x])[3])/2)
lbwart_tanzania['sex_ratio'] = np.where(lbwart_tanzania['sex']== 'Female', 0.487808, 1-0.487808)
lbwart_tanzania['weighted_bw'] = lbwart_tanzania['exposure'] * lbwart_tanzania['mean_bw'] * lbwart_tanzania['sex_ratio']

print('overall mean birthweight from artifact baseline (tanzania) = ', + lbwart_tanzania['weighted_bw'].sum())
print('overall mean birthweight from model baseline (tanzania) = ', + tanzania_bw_baseline['pop_x_bw'].sum()/100_000, )
print('overall mean sd of birthweight from model baseline (tanzania) = ', + tanzania_bwsd_baseline['pop_x_bw'].sum()/100_000)

lbwart_tanzania.head()

overall mean birthweight from artifact baseline (tanzania) =  3279.011285750329
overall mean birthweight from model baseline (tanzania) =  3274.7615355859452
overall mean sd of birthweight from model baseline (tanzania) =  636.1296742217747


,location,sex,age_start,age_end,year_start,year_end,parameter,exposure,affected_entity,affected_measure,relative_risk,mean_bw,sex_ratio,weighted_bw
0,Tanzania,Female,0.0,0.019178,2017.0,2018.0,cat10,0.001377,all,excess_mortality_rate,968.702386,750.0,0.487808,0.503795
1,Tanzania,Female,0.0,0.019178,2017.0,2018.0,cat106,0.000258,all,excess_mortality_rate,171.751901,1250.0,0.487808,0.157271
2,Tanzania,Female,0.0,0.019178,2017.0,2018.0,cat11,0.000293,all,excess_mortality_rate,513.870580,750.0,0.487808,0.107128
3,Tanzania,Female,0.0,0.019178,2017.0,2018.0,cat116,0.000464,all,excess_mortality_rate,182.396469,1250.0,0.487808,0.283021
4,Tanzania,Female,0.0,0.019178,2017.0,2018.0,cat117,0.002365,all,excess_mortality_rate,61.431240,1750.0,0.487808,2.019143
